In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
from matplotlib.patches import Rectangle
from matplotlib.ticker import MaxNLocator
from magnetar.funcs import init_conds, ODEs

%matplotlib inline

In [ ]:
# Global variables
G = 6.674e-8                    # Gravitational constant - cgs units
c = 3.0e8                       # Speed of light - cm/s
R = 1.0e6                       # Magnetar radius - km
Msol = 1.99e33                  # Solar mass - grams
M = 1.4 * Msol                  # Magnetar mass - grams
I = (4.0 / 5.0) * M * R ** 2.0  # Moment of Inertia
GM = G * M

In [ ]:
# Define the Bucciantini model
def bucc_ODEs(y, t, B, MdiscI, RdiscI, epsilon, delta, n, alpha=0.1, cs7=1.0,
              k=0.9):
    """
Define the set of ODEs to be solved by ODEINT using the dipole torque equation
defined in Bucciantini et al. (YYYY).

Usage >>> bucc_ODEs(y, t, B, MdiscI, RdiscI, epsilon, delta)
      y : output from init_conds
      t : array of time points to solve for
      B : Magnetic field strength - 10^15 Gauss (float)
 MdiscI : disc mass - Solar mass (float)
 RdiscI : disc radius - km (float)
epsilon : timescale ratio (float)
  delta : mass ratio (float)
      n : propeller "switch-on" efficiency (float)
  alpha : sound speed prescription (float)
    cs7 : sound speed - 10^7 cm/s (float)
      k : capping fration (float)

Returns and array object containing the time derivatives of the disc mass and
angular frequency to be integrated by ODEINT.
    """
    # Initial conditions
    Mdisc, omega = y
    
    # Constants
    Rdisc = RdiscI * 1.0e5                 # Convert disc radius to cm
    tvisc = Rdisc / (alpha * cs7 * 1.0e7)  # Viscous timescale
    mu = 1.0e15 * B * R ** 3.0             # Magnetic Dipole Moment
    M0 = delta * MdiscI * Msol             # Fallback Mass Budget
    tfb = epsilon * tvisc                  # Fallback timescale
    
    # Radii - Alfven, Corotation, Light Cylinder
    Rm = (mu ** (4.0 / 7.0) * GM ** (-1.0 / 7.0) * (Mdisc / tvisc) ** (-2.0 /
          7.0))
    Rc = (GM / omega ** 2.0) ** (1.0 / 3.0)
    Rlc = c / omega
    if Rm >= k * Rlc:
        Rm = k * Rlc
    
    w = (Rm / Rc) ** (3.0 / 2.0)   # Fastness Parameter
    bigT = 0.5 * I * omega ** 2.0  # Rotational energy
    modW = (0.6 * M * c ** 2.0 * ((GM / (R * c ** 2.0)) / (1.0 - 0.5 * (GM / (R
            * c ** 2.0)))))  # Binding energy
    rot_param = bigT / modW        # Rotation parameter
    
    # Dipole torque
    Ndip = ((-2.0 / 3.0) * ((mu ** 2.0 * omega ** 3.0) / c ** 3.0) * (Rlc / Rm)
            ** 3.0)
    
    # Efficiencies
    eta2 = 0.5 * (1.0 + np.tanh(n * (w - 1.0)))
    eta1 = 1.0 - eta2
    
    # Mass Flow Rates
    Mdotprop = eta2 * (Mdisc / tvisc)  # Propelled
    Mdotacc = eta1 * (Mdisc / tvisc)   # Accreted
    Mdotfb = (M0 /tfb) * (( t + tfb) / tfb) ** (-5.0 / 3.0)  # Fallback
    Mdotdisc = Mdotfb - Mdotprop - Mdotacc
    
    if rot_param > 0.27:
        Nacc = 0.0  # Prevents magnetar break-up
    else:
        # Accretion torque
        if Rm >= R:
            Nacc = (GM * Rm) ** 0.5 * (Mdotacc - Mdotprop)
        else:
            Nacc = (GM * R) ** 0.5 * (Mdotacc - Mdotprop)
    
    omegadot = (Nacc + Ndip) / I
    
    return np.array([Mdotdisc, omegadot])

In [ ]:
# Set up for calculations
alpha = 0.1  # Sound speed prescription
cs7 = 1.0    # Sound speed - 10^7 cm/s
k = 0.9      # Capping fraction

# Variables
B = 1.0          # Magnetic field strength - 10^15 G
P = 5.0          # Initial spin period - 5 milliseconds
MdiscI = 0.001   # Initial disc mass - solar masses
RdiscI = 1000.0  # Disc radius - km
epsilon = 0.1    # Timescale ratio
delta = 1.0      # Mass ratio
n = 10.0         # Propeller "switch-on"

# Conversions
Rdisc = RdiscI * 1.0e5                 # Convert disc radius to cm
tvisc = Rdisc / (alpha * cs7 * 1.0e7)  # Viscous timescale
mu = 1.0e15 * B * R ** 3.0             # Magnetic Dipole Moment
M0 = delta * MdiscI * Msol             # Fallback mass budget
tfb = epsilon * tvisc                  # Fallback timescale

modW = (0.6 * M * c ** 2.0 * ((GM / (R * c ** 2.0)) / (1.0 - 0.5 * (GM / (R * c
        ** 2.0)))))  # Binding energy

tarr = np.logspace(0.0, 6.0, num=10001, base=10.0)  # Time grid

y0 = init_conds(P, MdiscI)  # Initial conditions

In [ ]:
# Integrate Piro & Ott model
soln = odeint(ODEs, y0, tarr, args=(B, MdiscI, RdiscI, epsilon, delta, n))
Mdisc = soln[:,0]
omega = soln[:,1]

# Radii
Rm = mu ** (4.0 / 7.0) * GM ** ( -1.0 / 7.0) * (Mdisc / tvisc) ** (-2.0 / 7.0)
Rc = (GM / omega ** 2.0) ** (1.0 / 3.0)
Rlc = c / omega
Rm = np.where(Rm >= k * Rlc, k * Rlc, Rm)

w = (Rm / Rc) ** (3.0 / 2.0)   # Fastness parameter
bigT = 0.5 * I * omega ** 2.0  # Rotational energy
rot_param = bigT / modW        # Rotational parameter

Ndip = (-1.0 * mu ** 2.0 * omega ** 3.0) / (6.0 * c ** 3.0)  # Dipole torque

# Efficiencies and mass flow rates
eta2 = 0.5 * (1.0 + np.tanh(n * (w - 1.0)))
eta1 = 1.0 - eta2
Mdotprop = eta2 * (Mdisc / tvisc)
Mdotacc = eta1 * (Mdisc / tvisc)

# Accretion torque
Nacc = np.zeros_like(Ndip)
#Nacc = np.where(Rm >= R, (GM * Rm) ** 0.5 * (Mdotacc - Mdotprop), Nacc)
#Nacc = np.where(Rm < R, (GM * R) ** 0.5 * (Mdotacc - Mdotprop), Nacc)
#Nacc = np.where(rot_param > 0.27, 0.0, Nacc)
for i in range(len(Nacc)):
    if rot_param[i] > 0.27:
        Nacc[i] = 0.0  # Prevents magnetar break-up
    else:
        if Rm[i] >= R:
            Nacc[i] = (GM * Rm[i]) ** 0.5 * (Mdotacc[i] - Mdotprop[i])
        else:
            Nacc[i] = (GM * R) ** 0.5 * (Mdotacc[i] - Mdotprop[i])

# Luminosities
# Dipole luminosity
Ldip = (mu ** 2.0 * omega ** 4.0) / (6.0 * c ** 3.0)
Ldip = np.where(Ldip <= 0.0, 0.0, Ldip)
Ldip = np.where(np.isfinite(Ldip), Ldip, 0.0)

# Propeller luminosity
Lprop = ((-1.0 * Nacc * omega) - ((GM / Rm) * eta2 * (Mdisc / tvisc)))
Lprop = np.where(Lprop <= 0.0, 0.0, Lprop)
Lprop = np.where(np.isfinite(Lprop), Lprop, 0.0)

Ltot = Lprop + Ldip  # Total luminosity

In [ ]:
# Integrate Bucciantini model
bucc_soln = odeint(bucc_ODEs, y0, tarr, args=(B, MdiscI, RdiscI, epsilon,
                                              delta, n))
bucc_Mdisc = bucc_soln[:,0]
bucc_omega = bucc_soln[:,1]

"""
# Radii
bucc_Rm = (mu ** (4.0 / 7.0) * GM ** ( -1.0 / 7.0) * (bucc_Mdisc / tvisc) **
           (-2.0 / 7.0))
bucc_Rc = (GM / bucc_omega ** 2.0) ** (1.0 / 3.0)
bucc_Rlc = c / bucc_omega
bucc_Rm = np.where(bucc_Rm >= k * bucc_Rlc, k * bucc_Rlc, bucc_Rm)

bucc_w = (bucc_Rm / bucc_Rc) ** (3.0 / 2.0)   # Fastness parameter
bucc_bigT = 0.5 * I * bucc_omega ** 2.0  # Rotational energy
bucc_rot_param = bucc_bigT / modW        # Rotational parameter

# Dipole torque
bucc_Ndip = ((-2.0 / 3.0) * ((mu ** 2.0 * bucc_omega ** 3.0) / c ** 3.0) *
             (bucc_Rlc / bucc_Rm) ** 3.0)

# Efficiencies and mass flow rates
bucc_eta2 = 0.5 * (1.0 + np.tanh(n * (bucc_w - 1.0)))
bucc_eta1 = 1.0 - bucc_eta2
bucc_Mdotprop = bucc_eta2 * (bucc_Mdisc / tvisc)
bucc_Mdotacc = bucc_eta1 * (bucc_Mdisc / tvisc)

# Accretion torque
bucc_Nacc = np.zeros_like(bucc_Ndip)
bucc_Nacc = np.where(bucc_Rm >= R, (GM * bucc_Rm) ** 0.5 * (bucc_Mdotacc -
                     bucc_Mdotprop), bucc_Nacc)
bucc_Nacc = np.where(bucc_Rm < R, (GM * R) ** 0.5 * (bucc_Mdotacc -
                     bucc_Mdotprop), bucc_Nacc)
bucc_Nacc = np.where(bucc_rot_param > 0.27, 0.0, bucc_Nacc)

# Luminosities
# Dipole luminosity
bucc_Ldip = (mu ** 2.0 * bucc_omega ** 4.0) / (6.0 * c ** 3.0)
bucc_Ldip = np.where(bucc_Ldip <= 0.0, 0.0, bucc_Ldip)
bucc_Ldip = np.where(np.isfinite(bucc_Ldip), bucc_Ldip, 0.0)

# Propeller luminosity
bucc_Lprop = ((-1.0 * bucc_Nacc * bucc_omega) - ((GM / bucc_Rm) * bucc_eta2 *
              (bucc_Mdisc / tvisc)))
bucc_Lprop = np.where(bucc_Lprop <= 0.0, 0.0, bucc_Lprop)
bucc_Lprop = np.where(np.isfinite(bucc_Lprop), bucc_Lprop, 0.0)

bucc_Ltot = bucc_Lprop + bucc_Ldip  # Total luminosity
""";

In [ ]:
# Plotting
# Create a figure with 4 subplots
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(7,6))
leg1 = Rectangle((0, 0), 0, 0, alpha=0.0)  # Shape for legend

# Plot omega
ax1.semilogx(tarr, omega, c='k')
#ax1.semilogx(tarr, bucc_omega, c='k', ls='--')
ax1.set_xlim(1.0e0, 1.0e6)
ax1.yaxis.set_major_locator(MaxNLocator(5))
ax1.tick_params(axis='both', which='major', labelsize=10)
ax1.set_xlabel('Time (s)', fontsize=12)
ax1.set_ylabel('$\omega \left({\\rm s}^{-1}\\right)$', fontsize=12)
ax1.legend([leg1], ['(a)'], handlelength=0, loc=3, fontsize=10, frameon=False)

# Plot dipole torque
ax2.semilogx(tarr, Ndip, c='k')
ax2.set_xlim(1.0e0, 1.0e6)
ax2.yaxis.set_major_locator(MaxNLocator(5))
ax2.tick_params(axis='both', which='major', labelsize=10)
ax2.set_xlabel('Time (s)', fontsize=12)
ax2.set_ylabel('$N_{\\rm dip} ({\\rm erg/G cm^{3}})$ (solid line)',
               fontsize=12)
ax2.legend([leg1], ['(b)'], handlelength=0, loc=3, fontsize=10, frameon=False)

# Twin the y axis so bucc_Ndip can be plotted on different scale
#ax2twin = ax2.twinx()
#ax2twin.semilogx(tarr, bucc_Ndip, c='k', ls='--')
#ax2twin.tick_params(axis='both', which='major', labelsize=10)
#ax2twin.set_ylabel('$N_{\\rm dip} ({\\rm erg/G cm^{3}})$ (dashed line)',
#                   fontsize=12)

# Plot dipole luminosity
ax3.loglog(tarr, Ldip/1.0e50, c='k')
#ax3.loglog(tarr, bucc_Ldip/1.0e50, c='k', ls='--')
ax3.set_xlim(1.0e0, 1.0e6)
#ax3.set_ylim(1.0e-8, 1.0e0)
ax3.set_xticks([1.0e0, 1.0e2, 1.0e4, 1.0e6])
#ax3.set_yticks([1.0e-6, 1.0e-4, 1.0e-2, 1.0e0])
ax3.tick_params(axis='both', which='major', labelsize=10)
ax3.set_xlabel('Time (s)', fontsize=12)
ax3.set_ylabel('Dipole Luminosity ($10^{50} {\\rm erg/s}$)', fontsize=12)
ax3.legend([leg1], ['(c)'], handlelength=0, loc=3, fontsize=10, frameon=False)

# Plot total luminosity
ax4.loglog(tarr, Ltot/1.0e50, c='k')
#ax4.loglog(tarr, bucc_Ltot/1.0e50, c='k', ls='--')
ax4.set_xlim(1.0e0, 1.0e6)
#ax4.set_ylim(1.0e-8, 1.0e0)
ax4.set_xticks([1.0e0, 1.0e2, 1.0e4, 1.0e6])
#ax4.set_yticks([1.0e-6, 1.0e-4, 1.0e-2, 1.0e0])
ax4.tick_params(axis='both', which='major', labelsize=10)
ax4.set_xlabel('Time (s)', fontsize=12)
ax4.set_ylabel('Total Luminosity ($10^{50} {\\rm erg/s}$)', fontsize=12)
ax4.legend([leg1], ['(d)'], handlelength=0, loc=3, fontsize=10, frameon=False)

fig.tight_layout();